# Time Series Application — Scott Person's NOTEBOOK

**Applied Machine Learning 2 @ Newman University**

*Prof. Ricky Boyer*

**Jason "Scott" Person**

**Important note!** Before you turn in this lab notebook, make sure everything runs as expected:

- First, **restart the kernel** -- in the menubar, select Kernel$\rightarrow$Restart.
- Then **run all cells** -- in the menubar, select Cell$\rightarrow$Run All.

Make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`

# Part 0: Sample dataset (Finance)

The dataset we'll be using is accessible via a Python library called nasdaqdatalink. If the import cell does not run for you, then you may need to do a `pip` or `conda` install to access the data in your environment. Once you do, then restart the kernel and run the cells again. Cell added below to assist, or you can delete the cell and install the package via your typical `pip`. Friends using Colab should be able to use the below cell.

In [0]:
# Disable Databricks autolog - this ended up overloading the browser during stepwise regression
import mlflow
mlflow.autolog(disable=True)

In [0]:
%pip install nasdaq-data-link

In [0]:
# nasdaq data link for end of day stock price data
import nasdaqdatalink
import pandas as pd
import matplotlib.pyplot as plt
from numpy import isclose

#configure API: key intended only for student use while at Newman
nasdaqdatalink.ApiConfig.api_key = 'bzv_VKSXDstz5DLXnjhe'
txt = nasdaqdatalink.get_table('WIKI/PRICES', ticker = ['TXT'])
ba = nasdaqdatalink.get_table('WIKI/PRICES', ticker = ['BA'], paginate = True)
display(txt.head())
print("\n(All data appears to be ready.)")

In [0]:
txt.describe()

In [0]:
ba.describe()

Here we've brought in financial data for 2 aircraft companies, so we may get a sense of scale. Our main concern will be the prediction of Textron ([TXT](https://search.yahoo.com/search?fr=mcafee&type=E210US1590G0&p=Textron+stock)) market cap as if we were computing this back in 2018, using historical data. First let's take a look at the stock prices of these companies over their lifetime.

In [0]:
# The adjusted close accounts for stock splits, so that is what we should graph
plt.plot(ba.index, ba['close'])
plt.title('BA Stock Price')
plt.ylabel('Price ($)');
plt.show()
plt.plot(txt.index, txt['close'], 'r')
plt.title('TXT Stock Price')
plt.ylabel('Price ($)');
plt.show();

## Prepping Our Data
Generally speaking, for almost any date-based analysis, it is  agood idea to include a Date column. Ours happens to be in the index currently. We should remove it from the index using the `reset_index` function, but first we can put it to good use. Let's make a column `Year` using that index.

In [0]:
# Create a year column 
txt['Year'] = pd.DatetimeIndex(txt.index).year
ba['Year'] = pd.DatetimeIndex(ba.index).year

**Exercise 0** (10 points). Use the `reset_index` function, on our `txt` and `ba` DataFrames, with `level=0` and `inplace=True`. This should automatically give us a `Date` column in each of the datasets that virtually all time series models require, especially some of the newer ones like Prophet.


In [0]:
# Take Dates from index and move to Date column 
## YOUR ANSWER HERE
# BEGIN SOLUTION
txt.reset_index(level=0, inplace=True)
ba.reset_index(level=0, inplace=True)
# END SOLUTION

In [0]:
# Test cell: `rest_index_test`
assert txt['date'].dtype == 'datetime64[ns]', print("\n You do not have a 'Date' column in txt")
assert ba['date'].dtype == 'datetime64[ns]', print("\n You do not have a 'Date' column in ba")

print("\n(Passed! You got 10 points!)")

Now let's calculate the target that we are after. If trying to determine the Market Cap of a company, it is determined (in simplest form) by taking the outstanding shares of that company (represented by `adj_volume` in our dataset) and multiplying it by the given price of that share. Our dataset has many different prices represented for each day, but we'll use `close`.

**Exercise 1** (10 points). Create the column `Cap` in both the `txt` and `ba` datasets by multiplying the `adj_volume` with `close`.

In [0]:
## YOUR ANSWER HERE
# BEGIN SOLUTION
txt['Cap'] = txt['close'] * txt['adj_volume']
ba['Cap'] = ba['close'] * ba['adj_volume']

# END SOLUTION

In [0]:
txt.head()

In [0]:
# Test cell: `Cap`
assert txt['Cap'].dtype == 'float64', print("\n You do not have a 'Cap' column in txt or datatype wrong")
assert ba['Cap'].dtype == 'float64', print("\n You do not have a 'Cap' column in ba or datatype wrong")
assert txt['Cap'].shape == (8424,), print("\n Shape txt['Cap'] is {} when it should be {}").format(txt['Cap'].shape, '(8424,)')
assert ba['Cap'].shape == (14155,), print("\n Shape txt['Cap'] is {} when it should be {}").format(ba['Cap'].shape, '(14155,)')

print("\n(Passed! You got 10 points!)")

If we look at the two together, we can now see the scale of dollars that we are dealing with, as well as determine the general trend of our data.

In [0]:
mkt_cap = ba.merge(txt, how='inner', on='date')
mkt_cap.rename(columns={'Cap_x': 'ba_cap', 'Cap_y': 'txt_cap'}, inplace=True)
mkt_cap = mkt_cap.loc[mkt_cap['date'] > '2010-1-1', :]

plt.figure(figsize=(10, 8))
plt.plot(mkt_cap['date'], mkt_cap['ba_cap'], 'b-', label = 'BA')
plt.plot(mkt_cap['date'], mkt_cap['txt_cap'], 'r-', label = 'TXT')
plt.xlabel('date'); plt.ylabel('Market Cap (Billions $)'); plt.title('Market Cap of BA and TXT')
plt.legend();
mkt_cap = mkt_cap[['date', 'ba_cap', 'txt_cap']]
display(mkt_cap)

It appears that Boeing's markjet cap is much large than Textron's, though both can pretty easily be quantified in the Billions of dollars. We can also see the volatility in the day to day market. The hope would be that using our Time Series models has a smoothing effect over all that volatility. This view, however is much better than looking at it since the beginning of each company's entry onto the market.

**Exercise 2** (20 points). Slice the `txt` DataFrame such that you only keep the rows where `Date` is greater than '2010-1-1'. (**Hint:** If you look at my previous cell, you might be able to spot how I did it to the `mkt_cap` DataFrame.)

In [0]:
## YOUR ANSWER HERE
# BEGIN SOLUTION
txt = txt.loc[txt['date'] > '2010-1-1', :]
# END SOLUTION

In [0]:
# Test cell: `date_slice`
assert txt.shape == (2071, 17), "Shape txt is {} when it should be {}".format(txt.shape, '(2071, 17)')
try:
    test = txt.loc[txt['date'] < '2010-1-1', :]
    test.shape == (0,17)
    print("\n(Passed! You got 20 points!)")
except AssertionError as e:
    print("\n(Seems like you may still have some unwanted rows.)")

## Notation and review

Taking a look at the model explanations from [Toward Data Science](https://towardsdatascience.com/time-series-forecasting-with-arima-sarima-and-sarimax-ee61099e78f6) we can determine which model we may want to use, as well as a general explanation of how it works.

### Autoregression (AR)
**Autoregression** utilizes a $p$ parameter to determine how many lag variables we use in the regression to determine the next value, according to the following formula. Setting this value to 1 begins to introduce randomness into the algorithm, shifting away from a normal regression. 

![image.png](attachment:839d9674-4f7a-4cb5-824d-90711f2a450a.png)

### Moving Average (MA)
The **Moving Average** portion of time series algorithms utilizes a $q$ parameter to determine how many lag variables to use when determining the average. In an $q$ = 1 model, our forecast is a constant term plus the previous white noise term times a multiplier, added with the current white noise term.

### ARMA
The **ARMA** model is the combination of the previous two sections in one algorithm, acting as the basic framework for most TS analyses used in business today.

### ARIMA
**ARIMA** utilizes the ARMA model and a difference order as defined by $I(d)$.

### SARIMA
**SARIMA** is an ARIMA model that has additional ability to difference based on seasonality as defined within the parameters of the formula. It expressed as a function is shown belw, which is a faar cry from the original AR model.

![image.png](attachment:25264769-4bbd-4e57-b3df-ec7ac042768a.png)

For our purposes, both the ARIMA and SARIMA models can be accessed via our friendly [Statsmodels package](https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima.model.ARIMA.html) from the lesson on Regression! Let's go ahead and pull in that model as well as some of it' visualization tools.

In [0]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_predict

# Fitting a model

We've arrived at the main event! Let's check out what this looks like when we run a very basic model where $(p, d, q)$ are all set to 0 and we have no seasonality introduced. This should allow us to check out the baseline summary statistics.

In [0]:
# 0,0,0 ARIMA Model
model = ARIMA(txt.Cap, order=(0,0,0), seasonal_order=(0,0,0,0))
model_fit = model.fit()
print(model_fit.summary())

We want to make sure each term in our model is statistically significant, which we can see by taking a look at the highlighted section. We want each term to have a p-value of less than 0.05, so we can reject the null hypothesis with statistically significant values.

![image.png](attachment:02ba54a5-7509-44ef-baa4-24d84f3b6c22.png)

We can see that the Constant value is way off from the .05 or less that we need it to be in order to show statistical significance.

**Let's get Experimental!** Like most of our algorithms, the feature engineering is what tends to produce better results within the output models, however time series analysis allows us to adjust the regressor variables in real time as hyperparameters of the algorithm. It seems like a good idea to play around with the parameters a bit to see if we can get something a little better.


**Exercise 3** (20 points). Copy and paste my ARIMA code cell into the next code cell, then adjust `order` parameter to be $(p, d, q)$ = (3, 1, 3). Keep the `model` and `model_fit` variables as they are.

In [0]:
# 3,1,3 ARIMA Model
##YOUR CODE HERE
#BEGIN SOLUTION
model = ARIMA(txt.Cap, order=(3,1,3), seasonal_order=(0,0,0,0))
model_fit = model.fit()
#END SOLUTION
print(model_fit.summary())

In [0]:
# Test cell: `313_arima`

test = [-1.442948421186277, -0.20025750284682942, 0.4024901151346325, 0.8691307148401701, -0.840341457052922, -0.9281957313898933, 1791497880191682.8]
#assert test == list(model_fit.params), "Check your cell again. It should look exactly like before with order = (3 ,1 ,3)"
assert all(isclose(list(test), list(model_fit.params), rtol=.001)), "Check your cell again. It should look exactly like before with order = (3 ,1 ,3)"
print("\n(Passed! You got 20 points!)")

From a variable standpoint this looks much better as all seem to have a $p$ value lower than .05. However, when we check the statistics based on our normal regression assumtions, we now see low heteroskedasticity, as well as high $p$ values for the Ljun-Box test and the heteroskedasticity. this indicates that we cannot rejcet the assumed state of violating those assumptions. 

![image.png](attachment:80283e74-56f3-451c-92a2-d6f71d4aa4f3.png)

Can adding in the `seasonal_order` parameter help us get on track?

**Exercise 4** (30 points). Copy and paste the ARIMA code cell into the next code cell. Keep the `order` parameter as $(p, d, q)$ = (3, 1, 3) and the `model` and `model_fit` variables as they are. Then go ahead and play with the `seasonal_order` parameter, setting it to something other than $(P, D, Q, s)$ = (0, 0, 0, 0). Within the model fit method you may also need to increase the `method_kwargs` to ensure convergance of the model.

In [0]:
# Do not execute 
'''
# 1,1,0 ARIMA Model
##YOUR CODE HERE
#BEGIN SOLUTION

# 20250604 JSP OK, I monkeyed with the pqds values. Just wandering around in the dark on this one.

model = ARIMA(txt.Cap, order=(3,1,3), seasonal_order=(0,1,1,20)) # 1,1,1,365 is blows up the cluster I went with 20 because that is monthly sasonality (for work days). Just guessing here.
model_fit = model.fit(method_kwargs={"maxiter": 200})
#END SOLUTION

print(model_fit.summary())
'''

In [0]:
# Test cell: `sarima`
#Yay a free 20 points for getting the above to run! You'll all have something different, but as long as it runs you'll get the points.
print("\n(Passed! You got 20 points!)")

In [0]:
# Plot residual errors
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [0]:
# Actual vs Fitted
plot_predict(model_fit, dynamic=False)
plt.show()

# Other Algorithms and Packages
There are a few more packages that we would be a little remissed if we did not talk about:

### Auto ARIMA
Auto arima is part of the [pmdarima](https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.auto_arima.html) package that allows for optimized and iterable runs of the ARIMA function through a helpful configuration function. It allows you to set a `start_p`, `max_p`, `start_q`, and `max_q` parameters so it can iterate through each model, determining which best fits the test given. It also has a few more advanced regression parameters like the ability to run the model stepwise in considering each regression variable.

### Prophet
[Prophet](https://facebook.github.io/prophet/docs/quick_start.html) is the time series algorithm developed by Meta (formaerly Facebook) and is widely considered one of the better time series models out there right now. It is also built in such a way that makes predictions very easy to accomplish, necessitating only a few lines of code. The only finnicky thing about it is that it requires a `ds` field (in the form of a date series) and a `y`

Let's go ahead and install those packages. You may need to `!pip` or `conda` install them to get them to work.

In [0]:
#%pip install pmdarima
#%pip install prophet

In [0]:
import pmdarima as pm
from prophet import Prophet

Let's see if the automated version can generate a better model than earlier. **DISCLAIMER: This may take a while to run.**

In [0]:
#Standard ARIMA Model
ARIMA_model = pm.auto_arima(txt['Cap'], 
                      start_p=0, 
                      start_q=0,
                      test='kpss', #default
                      max_p=3, max_q=3, # maximum p and q
                      m=7, # frequency of series (if m==1, seasonal is set to FALSE automatically)
                      d=None,# let model determine 'd'
                      seasonal=True, # No Seasonality for standard ARIMA
                      trace=False, #logs
                      error_action='warn', #shows errors ('ignore' silences these)
                      suppress_warnings=True,
                      stepwise=True)

print(ARIMA_model.summary())

ARIMA_model.plot_diagnostics(figsize=(15,12))
plt.show()

It seems that our earlier $(p, d, q)$ = (3,1,3) is about as good as it is going to get for this data. Each overall coefficient in the regression is statistically significant even if there is some imbalance overall in the dataset. Time series are much harder to balance than our priior classification models since they require a value for each date in the series. Other options at this point would be restructuring our dataframe to be organized by week or month. Overall, though, for a business something like this still helps us determine the regressive components of the market cap, allowing us to glean insight from the selected model itself, prior to any forecasting.

Let's do a little forecasting with the [Prophet](https://facebook.github.io/prophet/docs/quick_start.html). Open up the documentation, and let's walk through the first few steps.

**Exercise 5** (30 points). Complete the checklist:
* Create DataFrame `txt` by renaming the current column `date` to `ds` and the column `close` to `y`
* Create `txt_prophet` by instantiating a new `Prophet()` object, but let's include `changepoint_prior_scale=0.15` in the Prophet configuration.
* Fit the `txt_prophet` object to our intial `txt` DataFrame

In [0]:

##YOUR ANSWER HERE
#BEGIN SOLUTION
# Prophet requires columns ds (Date) and y (value)
txt = txt[['date', 'Cap']].rename(columns={'date': 'ds', 'Cap': 'y'})
# Make the prophet model and fit on the data
txt_prophet = Prophet()
txt_prophet.fit(txt)
#END SOLUTION


In [0]:
# Test cell: `prophet_prep`
try: 
    test1 = txt['date']
except KeyError:
    pass
except AttributeError:
    pass
except NameError:
    pass
try:    
    print(txt_prophet)
except KeyError:
    print('txt_prophet does not exist')
except AttributeError:
    print('txt_prophet does not exist')
except NameError:
    print('txt_prophet does not exist')

print("\n(Passed! You got 30 points!)")

**Exercise 6** (20 points). Complete the checklist:
* Create DataFrame `txt_forecast` by using the `make_future_dataframe` function for 2 years worth of days (periods= 365*2, freq='D')
* Use the `.predict` method from our `txt_prophet` on our newly created `txt_forecast` DataFrame.

In [0]:
##YOUR ANSWER HERE
#BEGIN SOLUTION
# Make a future dataframe for 2 years
txt_forecast = txt_prophet.make_future_dataframe(periods=365*2, freq='D')

# Make predictions
txt_forecast = txt_prophet.predict(txt_forecast)
#END SOLUTION


In [0]:
# Test cell: `prophet_predict`

try:    
    prophet_predict_test = txt_forecast.iloc[:, [1, 2, 5]]
except AttributeError:
    print('Something is wrong with your prediction')

assert list(prophet_predict_test.columns) == ['trend', 'yhat_lower', 'trend_upper'], print('Something is wrong with your prediction')

print("\n(Passed! You got 30 points!)")

In [0]:
txt_prophet.plot(txt_forecast, xlabel = 'Date', ylabel = 'Market Cap (billions $)')
plt.title('Market Cap of TXT');

Prophet can have its hyperparameters tuned as well! We can iterate through a bunch of hyperparameters with the model, by forming a matrix. We can then iterate over our model, using the permuations within the matrix. This will tell us what the best hyperparameters are! For the long-hand version of this, you can check out the [Prophet Diagnostics Documentation](https://facebook.github.io/prophet/docs/diagnostics.html) this part of the notebook is based on. This will take a little to run as well, since it is running through a number of models. Give it a few minutes.

In [0]:
import itertools
import numpy as np
import pandas as pd
from prophet.diagnostics import cross_validation, performance_metrics

param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params).fit(txt)  # Fit model with given params
    df_cv = cross_validation(m, horizon='30 days', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

**Exercise 7** (20 points). Input the values for `changepoint_prior_scale` as variable `cps` and `seasonality_prior_scale` as variable `sps` that create the best model best on the above table. You can also be clever by using the `argmin` function in the Numpy package.


In [0]:
index = tuning_results['rmse'].idxmin()

cps = tuning_results.iloc[index]['changepoint_prior_scale']
sps = tuning_results.iloc[index]['seasonality_prior_scale']

In [0]:
# Test cell: `txt_hyp_test`
sanswer = [cps, sps]
min_rmse = tuning_results['rmse'].min()

best_params = tuning_results[tuning_results['rmse'] == min_rmse]
best_params = best_params.drop(columns=['rmse']).iloc[0].tolist()
assert best_params == sanswer, print('Try a different set of hyperparameters')

print("\n(Passed! You got 20 points!)")

**Congrats!** If you've gotten this far without errors, you're ready to submit your notebook!